In [99]:
#pip install ucimlrepo-- UNCOMMENT


In [ ]:
#!pip install prettytable

In [14]:
#pip install pandas-- UNCOMMENT

In [15]:
#pip install scikit-learn-- UNCOMMENT

In [ ]:
# WE CAN ADD SOME INTRO/BACKGROUND FROM THIS PAPER 

# https://academic.oup.com/gigascience/article/9/11/giaa128/6006352 



Relevant snippet copied from paper:

Previously, the dataset was a subject of machine learning method applications, including convolutional neural networks [23] and dimensionality reduction methods [24].


As a first step of pre-processing, 7 variables were removed from the initial MI complication data table as containing >30% of missing values. Next, 126 records were removed as containing >20% of the missing values. After this step, the data table contained 2.5% of missing values with 533 rows (34% of all clinical cases) having no missing values.

After the missing value filtering step, the data table of MI complications contained 84 binary, 9 continuous numerical, 22 ordinal, and 1 categorical variables. The large number of ordinal variables requires careful quantification (see Methods), which is not trivial given the large number of rows with missing values.

We considered the number of continuous numerical variables too small to apply the methodology of categorical principal component analysis (CatPCA) [28]. Therefore, for all ordinal and binary variables we first applied univariate quantification following the approach described in the Methods section. This quantification allowed the application of the “SVDComplete" imputation method for imputing the missing values, as described in the Methods. After all missing values were imputed, we could apply the optimal scaling approach for ordinal values, optimizing the pairwise correlations between them and between ordinal and continuous numerical variables. The 22 ordinal variables quantified in this way were further used for forming the data space. In addition, all variables were converted to z-scores.


Note: features already ordinally encoded

# Setup

In [100]:
# READ IN DATA AND IMPORT RELEVANT PACKAGES

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from prettytable import PrettyTable
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight


  
# fetch dataset 
myocardial_infarction_complications = fetch_ucirepo(id=579) 
  
# data (as pandas dataframes) 
X = myocardial_infarction_complications.data.features 
y = myocardial_infarction_complications.data.targets 


In [2]:
# GET RID OF FEATURES AND OBSERVATIONS WITH A LOT OF MISSINGNESS

# first, find percentage of missing values
missing_percentage = (X.isna().sum() / len(X)) * 100

# List of columns with over 30% missing values
columns_to_remove = missing_percentage[missing_percentage > 30].index
X = X.drop(columns = columns_to_remove)

# we will also remove rows with >20% missingess
missing_percentage_per_row = (X.isna().sum(axis=1) / X.shape[1]) * 100
rows_to_remove = missing_percentage_per_row[missing_percentage_per_row > 20].index

# Remove the identified rows from both X and Y
X = X.drop(index=rows_to_remove)
y = y.drop(index=rows_to_remove)

# TODO: Explain this process in milestone and report the percentage of missingness in the dataframe, how many features were removed, etc.

In [252]:
# CREATE NEW OUTCOME THAT IS 1 IF ANY EVENT OCCURS AND 0 IF NO EVENT OCCURS
events = ['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK',
       'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN']
y['any_event'] = y[events].any(axis=1).astype(int)


# CREATE NEW OUTCOME THAT GROUPS CAUSES OF DEATH TOGETHER SO WE HAVE LESS OUTCOMES
# INSPO FROM CHATGPT (SEE BELOW)

def map_to_category(cause):
    if cause in [1, 3, 4]:
        return 'cardiac_complications'
    elif cause in [2, 5]:
        return 'pulmonary_complications'
    elif cause in [6, 7]:
        return 'cardiac_arrest'
    else:
        return 'alive' 

y['cause_of_death'] = y['LET_IS'].apply(map_to_category)

/var/folders/0d/w3zvknms6yvf104977q75_jm0000gn/T/ipykernel_68607/1724817802.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['any_event'] = y[events].any(axis=1).astype(int)
/var/folders/0d/w3zvknms6yvf104977q75_jm0000gn/T/ipykernel_68607/1724817802.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['cause_of_death'] = y['LET_IS'].apply(map_to_category)


In [39]:
# SPLIT DATA INTO TRAIN AND TEST

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

# Determine which features are categorical (ordinal), continuous, and binary
# might be helpful later

unique_non_missing_values = X.nunique(dropna=True)
unique_non_missing_values_df = unique_non_missing_values.reset_index()
unique_non_missing_values_df.columns = ['feature', 'n_unique_values']


continuous_features = ['AGE', 'D_AD_ORIT', 'S_AD_ORIT', 'K_BLOOD', 
                       'NA_BLOOD', 'ALT_BLOOD', 'AST_BLOOD', 'L_BLOOD', 
                       'ROE']

binary_features = list(unique_non_missing_values_df[(unique_non_missing_values_df['n_unique_values'] == 2) & (~unique_non_missing_values_df['feature'].isin(continuous_features))]['feature'])

ordinal_features = list(unique_non_missing_values_df[(unique_non_missing_values_df['n_unique_values'] > 2) & (~unique_non_missing_values_df['feature'].isin(continuous_features))]['feature'])


# TODO: Note in paper that we split our data into training and testing

In [67]:
# CHECK BALANCE IN TRAIN AND TEST SPLITS FOR 'ANY_EVENT' OUTCOME

print(sum(y_train['any_event'])/len(y_train))
print(sum(y_test['any_event'])/len(y_test))

# Both around 50%! We probably do not need to rebalance for the binary classification task


# CHECK BALANCE IN TRAIN AND TEST SPLITS FOR 'CAUSE OF DEATH' OUTCOME
print(y_train['LET_IS'].value_counts(normalize=True))
print(y_test['LET_IS'].value_counts(normalize=True))

# we see a much greater degree of imbalance in this column, with around 85% 
# alive outcomes and only 15% being any other cause of death

# TODO: In paper note the degree of balance/imbalance in all of our possible outcomes
# TODO: Make these into nice tables for the paper 

0.5406392694063927
0.5574468085106383
LET_IS
0    0.852055
1    0.054795
3    0.028311
7    0.018265
6    0.014612
4    0.012785
2    0.010959
5    0.008219
Name: proportion, dtype: float64
LET_IS
0    0.825532
1    0.070213
3    0.046809
6    0.014894
4    0.014894
7    0.010638
2    0.010638
5    0.006383
Name: proportion, dtype: float64


In [251]:
# For individual outcomes, want to look at proportion of outcome vs. no outcome as well
y_outcomes = y_train[events]
proportions = (y_outcomes.sum() / len(y_outcomes)).round(2)  
print(proportions)

# TODO: Report this in fancy table in paper
# Also include a table that shows the number/proportion of overlaps between these

FIBR_PREDS    0.10
PREDS_TAH     0.01
JELUD_TAH     0.03
FIBR_JELUD    0.05
A_V_BLOK      0.03
OTEK_LANC     0.10
RAZRIV        0.03
DRESSLER      0.04
ZSN           0.24
REC_IM        0.08
P_IM_STEN     0.08
dtype: float64


In [83]:
# IMPUTE MISSING VALUES ON TRAINING AND TESTING DATA
# using simple KNN imputer for now

def impute_values(X, n):
    imputer = KNNImputer(n_neighbors=n)
    imputed = imputer.fit_transform(X)
    return imputed

# Fit the imputer on the training data and transform train and test data
X_train_imputed = impute_values(X_train, 5)
X_test_imputed = impute_values(X_test, 5)

#TODO: Explain this simple process in paper and note that without more knowledge on missingness we will not explore other methods


# Binary Classification with Logistic Regression

We are predicting the target of any event vs. no event. These are quite balanced outcomes (see percentages above!) so we will not deal with rebalancing the data.

In [84]:
# TRAIN LOGISTIC REGRESSION MODEL WITH LASSO 

# train model
model = LogisticRegression(penalty = "l1", solver='liblinear')
model.fit(X_train_imputed, y_train['any_event'])

# make predictions on test data
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['any_event'], y_pred)
print(accuracy)
confusion_matrix(y_test['any_event'], y_pred)

# we have accuracy of almost 70% 
# can we format this confusion matrix better?
# we can make note that accuracy feels reasonable to look at here because of class balance



0.6829787234042554


array([[140,  68],
       [ 81, 181]])

In [87]:
# FIGURE OUT WHICH FEATURES REMAINED IN THE MODEL AFTER LASSO

coefficients = model.coef_
feature_names = X_train.columns  
feature_coefficients = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients[0]})
selected_features = feature_coefficients[feature_coefficients['Coefficient'] != 0]
selected_features

# 65 features remain of original 104! 

# TODO: Explain this process in paper and print out classificaiton report
# Note that we will also try this with a deep learning model to see if we can bump up accuracy

,Feature,Coefficient
0,AGE,0.031181
1,SEX,-0.199716
3,STENOK_AN,0.028116
4,FK_STENOK,-0.060182
5,IBS_POST,0.092338
...,...,...
97,LID_S_n,0.313669
99,ANT_CA_S_n,-0.015915
100,GEPAR_S_n,0.141909
101,ASP_S_n,0.008445


# Multi-Class Classification with Logistic Regression

We are now predicting the cause of death outcome, where as you can recall there is a great deal of class imbalance. In the training set 85% of outcomes are alive, while the remainining 15% is split between 7 different options.

In writeup describe what all of these causes are.

We will first try to just run a logistic regression without accounting for class imbalance at all. We will use the l1 regularization again.

There are two options, one v. one and one v. rest. In the OvR strategy, you train a separate binary classifier for each class, treating it as the positive class, while considering all other classes as the negative class.

OvR is often preferred when you have a large number of classes, and it can be more computationally efficient.
It's suitable for imbalanced datasets and is less sensitive to imbalanced class distributions.
OvR can handle both binary and multiclass classification problems without modification. 

We will start with OVR. 





### Crude: No Balancing

In [108]:
# train model
model = LogisticRegression(multi_class='ovr', solver='liblinear', penalty = 'l1')
model.fit(X_train_imputed, y_train['LET_IS'])
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['LET_IS'], y_pred)
report = classification_report(y_test['LET_IS'], y_pred)

/Users/malaviravindran/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/malaviravindran/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/malaviravindran/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [103]:

report_data = report.split('\n')
table = PrettyTable(["Category"] + report_data[0].split())
for line in report_data[2:-5]:
    table.add_row(line.split())
print(table)

+----------+-----------+--------+----------+---------+
| Category | precision | recall | f1-score | support |
+----------+-----------+--------+----------+---------+
|    0     |    0.85   |  1.00  |   0.92   |   388   |
|    1     |    0.91   |  0.30  |   0.45   |    33   |
|    2     |    0.00   |  0.00  |   0.00   |    5    |
|    3     |    0.25   |  0.05  |   0.08   |    22   |
|    4     |    0.00   |  0.00  |   0.00   |    7    |
|    5     |    0.00   |  0.00  |   0.00   |    3    |
|    6     |    0.00   |  0.00  |   0.00   |    7    |
|    7     |    0.00   |  0.00  |   0.00   |    5    |
+----------+-----------+--------+----------+---------+


TODO: Figure out which metric we care about in the imbalanced setting??

### Re-Balancing Training Data

We can try this in a few ways ways:

1. Upsampling minority classes
2. Class Weighted Loss

First we will start simple with oversampling minority classes. 

#### Oversampling minority classes

We will oversample minority classes to the size of the original data

In [165]:
class_counts = y_train['LET_IS'].value_counts()
minority_classes = class_counts[class_counts != class_counts.max()].index
columns = list(X_train.columns) + list(y_train.columns)
upsampled_data = pd.DataFrame(columns=columns)
combined_df = pd.DataFrame(np.hstack((X_train_imputed, np.array(y_train))), columns = columns)

for minority_class in minority_classes:
    # Subset the minority class instances
    minority_data = combined_df[combined_df['LET_IS'] == minority_class]

    # Oversample the minority class to match the size of the majority class
    minority_upsampled = resample(minority_data, replace=True, n_samples=class_counts.max(), random_state=42)

    # Add the upsampled data to the DataFrame
    upsampled_data = pd.concat([upsampled_data, minority_upsampled])

# Combine the upsampled data with the majority class data
balanced_data = pd.concat([combined_df[combined_df['LET_IS'] == 0], upsampled_data])

# Separate the features and target variable from the upsampled data
X_train_upsampled = np.array(balanced_data.drop(y_train.columns, axis=1))
y_train_upsampled = balanced_data[y_train.columns]




/var/folders/0d/w3zvknms6yvf104977q75_jm0000gn/T/ipykernel_68607/1917138588.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  upsampled_data = pd.concat([upsampled_data, minority_upsampled])


In [167]:
# train model and make predictions as usual
model = LogisticRegression(multi_class='ovr', solver='liblinear', penalty = 'l1')
model.fit(X_train_upsampled, y_train_upsampled[])
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['LET_IS'], y_pred)
report = classification_report(y_test['LET_IS'], y_pred)

In [168]:
# our accuracy takes a hit, but we have better recall in four more classes. 
print(accuracy)
report_data = report.split('\n')
table = PrettyTable(["Category"] + report_data[0].split())
for line in report_data[2:-5]:
    table.add_row(line.split())
print(table)

0.6340425531914894
+----------+-----------+--------+----------+---------+
| Category | precision | recall | f1-score | support |
+----------+-----------+--------+----------+---------+
|    0     |    0.92   |  0.71  |   0.80   |   388   |
|    1     |    0.33   |  0.45  |   0.38   |    33   |
|    2     |    0.10   |  0.20  |   0.13   |    5    |
|    3     |    0.10   |  0.18  |   0.12   |    22   |
|    4     |    0.08   |  0.14  |   0.11   |    7    |
|    5     |    0.00   |  0.00  |   0.00   |    3    |
|    6     |    0.00   |  0.00  |   0.00   |    7    |
|    7     |    0.00   |  0.00  |   0.00   |    5    |
+----------+-----------+--------+----------+---------+


#### Reweighting Classes

We can also try to assign different weights to classes to give more importance to minority classes. 

In [171]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train['LET_IS']), y=y_train['LET_IS'])

# Create a dictionary with class weights
class_weight_dict = dict(enumerate(class_weights))

# train model and make predictions as usual
model = LogisticRegression(multi_class='ovr', solver='liblinear', penalty = 'l1', class_weight=class_weight_dict)
model.fit(X_train_imputed, y_train['LET_IS'])
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['LET_IS'], y_pred)
report = classification_report(y_test['LET_IS'], y_pred)

In [172]:
# our accuracy is better than before, but still poor performance on the smaller classes 
print(accuracy)
report_data = report.split('\n')
table = PrettyTable(["Category"] + report_data[0].split())
for line in report_data[2:-5]:
    table.add_row(line.split())
print(table)

0.7574468085106383
+----------+-----------+--------+----------+---------+
| Category | precision | recall | f1-score | support |
+----------+-----------+--------+----------+---------+
|    0     |    0.91   |  0.87  |   0.89   |   388   |
|    1     |    0.52   |  0.39  |   0.45   |    33   |
|    2     |    0.10   |  0.20  |   0.13   |    5    |
|    3     |    0.22   |  0.23  |   0.22   |    22   |
|    4     |    0.00   |  0.00  |   0.00   |    7    |
|    5     |    0.00   |  0.00  |   0.00   |    3    |
|    6     |    0.00   |  0.00  |   0.00   |    7    |
|    7     |    0.00   |  0.00  |   0.00   |    5    |
+----------+-----------+--------+----------+---------+


### Consolidating Classes


We can also try to consolidate our classes so that we have more manageable outcomes. According to chatgpt:

Cardiac Complications:
Cardiogenic shock (1)
Myocardial rupture (3)
Progress of congestive heart failure (4)
These conditions are all related to the function and complications of the heart. They could be grouped together as "Cardiac Complications."

Pulmonary/Cardiopulmonary Complications:
Pulmonary edema (2)
Thromboembolism (5)
Both of these conditions involve complications in the pulmonary system or the interaction between the heart and the lungs.

Cardiac Arrest:
Asystole (6)
Ventricular fibrillation (7)
Both asystole and ventricular fibrillation are types of cardiac arrest. You could group them together as "Cardiac Arrest."


We will upsample our data according to these four classes.



In [240]:
class_counts = y_train['cause_of_death'].value_counts()
minority_classes = class_counts[class_counts != class_counts.max()].index
columns = list(X_train.columns) + list(y_train.columns)
upsampled_data = pd.DataFrame(columns=columns)
combined_df = pd.DataFrame(np.hstack((X_train_imputed, np.array(y_train))), columns = columns)

for minority_class in minority_classes:
    # Subset the minority class instances
    minority_data = combined_df[combined_df['cause_of_death'] == minority_class]

    # Oversample the minority class to match the size of the majority class
    minority_upsampled = resample(minority_data, replace=True, n_samples=class_counts.max(), random_state=42)

    # Add the upsampled data to the DataFrame
    upsampled_data = pd.concat([upsampled_data, minority_upsampled])

# Combine the upsampled data with the majority class data
balanced_data = pd.concat([combined_df[combined_df['cause_of_death'] == 'alive'], upsampled_data])

# Separate the features and target variable from the upsampled data
X_train_upsampled = np.array(balanced_data.drop(y_train.columns, axis=1))
y_train_upsampled = balanced_data[y_train.columns]


# our accuracy takes a hit, but we have better recall in four more classes. 
# train model and make predictions as usual
model = LogisticRegression(multi_class='ovr', solver='liblinear', penalty = 'l1')
model.fit(X_train_upsampled, y_train_upsampled['cause_of_death'])
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['cause_of_death'], y_pred)
report = classification_report(y_test['cause_of_death'], y_pred)


In [241]:
# understandably, we are still performing poorly on the lower classes, but we should consider the support.
# there are so few observations in these classes still. 
print(accuracy)
report_data = report.split('\n')
table = PrettyTable(["Category"] + report_data[0].split())
for line in report_data[2:-5]:
    table.add_row(line.split())
print(table)

0.6702127659574468
+-------------------------+-----------+--------+----------+---------+
|         Category        | precision | recall | f1-score | support |
+-------------------------+-----------+--------+----------+---------+
|          alive          |    0.94   |  0.73  |   0.82   |   388   |
|      cardiac_arrest     |    0.03   |  0.17  |   0.05   |    12   |
|  cardiac_complications  |    0.38   |  0.45  |   0.41   |    62   |
| pulmonary_complications |    0.04   |  0.12  |   0.06   |    8    |
+-------------------------+-----------+--------+----------+---------+


# Remaining Tasks

1. Write out intro, describing our major problems of interest and what we plan to do to address each of these. 
2. Describe dataset features (time of collection), targets of interest, and initial dataset preprocessing (we should probably look into what these targets actually are)
3. Plots/Tables: Percentage of various outcomes, table showing amount of overlap between possible outcomes, maybe some plots of outcome vs. feature
4. Simple binary classification with logistic regression-- explain how we did lasso for convergence and report confusion matrix (formatted nicely). Say that we have a decent accuracy and that our outcomes are quite balanced here. But we can do better in terms of accuracy and our next steps in the binary classification setting are to explore 
5. Then say that we also have a multi-class classification problem of interest, where imbalance is a much greater problem. Talk about the specific setup and what metrics we will need to use. We have so far tried 1) no rebalancing, 2) upsampling, 3) reweighting, and 4) consolidating classes and upsampling again. So far doesn't look like we do great in any case so this data just might not be ideal. So for our next steps, we are going to turn to either 1) just predicting this as a binary outcome or 2) using multi-label classification to predict multiple outcomes for a patient. For these we will again try simple LR and then move over to deep learning techniques. If we do multi-label then we may just select a few relevant ones instead of 13 (or however many there are). 

And organize everything into:

Motivation: What problem are you tackling, and what's the setting you're considering?
Method: What machine learning techniques have you tried and why?
Preliminary experiments: Describe the experiments that you've run, the outcomes, and any error analysis that you've done. You should have tried at least one baseline.
Next steps: Given your preliminary results, what are the next steps that you're considering?


# Code Backlog -- THINGS I TRIED THAT DIDN'T DO ANYTHING

In [ ]:
# Let's manually do cross validation to see if we can find a good 
# combination of C (lasso regularization parameter) and n (KNN parameter)
# Can make a note that we tried this but it made very little difference


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

ns = [1, 3, 5, 10, 15, 20, 30, 50]
cs = [0.001, 0.01, 0.1, 1, 10]
n_repeats = 3

# Initialize a list to store results
results = []

def manual_cv_logreg(X, y, ns, cs, n_repeats): 
    for n in ns:
        for c in cs:
            print(f"n: {n}, c: {c}")    
            repeat_results = []  
            for repeat in range(n_repeats):
                #print(f"Repeat {repeat + 1}/{n_repeats}")
    
                # Create a KFold cross-validator with 'n_splits' folds
                kf = KFold(n_splits=3, shuffle=True, random_state=42)
    
                # Initialize a list to store the cross-validation results for this repeat
                fold_results = []
    
                for train_idx, test_idx in kf.split(X):
                    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                    # impute 
                    X_train_imputed = impute_values(X_train, n)
                    X_test_imputed = impute_values(X_test, n)

                    #train model
                    model = LogisticRegression(penalty = "l1", solver='liblinear')
                    model.fit(X_train_imputed, y_train)

                    # make predictions on test data
                    y_pred = model.predict(X_test_imputed)
    
                    # Calculate the accuracy or other evaluation metric
                    accuracy = accuracy_score(y_test, y_pred)
                    fold_results.append(accuracy)
    
                # Calculate the mean accuracy and standard deviation for this repeat
                mean_accuracy = sum(fold_results) / len(fold_results)
                std_accuracy = (sum((x - mean_accuracy) ** 2 for x in fold_results) / len(fold_results)) ** 0.5
    
                #print(f"Mean Accuracy: {mean_accuracy:.4f}")
                #print(f"Std. Deviation: {std_accuracy:.4f}\n")
    
                repeat_results.append(mean_accuracy)
    
            # Calculate the overall mean accuracy across repeats for this parameter combination
            overall_mean_accuracy = sum(repeat_results) / len(repeat_results)
            results.append((n, c, overall_mean_accuracy))
    
    # Find the best parameter combination
    best_params = max(results, key=lambda x: x[2])
    print(f"Best Parameters: Parameter 1 = {best_params[0]}, Parameter 2 = {best_params[1]}")
    return best_params


manual_cv_logreg(X_train, y_train['any_event'], ns, cs, n_repeats)

In [ ]:
# testing out a random forest model???
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train_upsampled, y_train_upsampled['LET_IS'])
y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test['LET_IS'], y_pred)
report = classification_report(y_test['LET_IS'], y_pred)

# our accuracy takes a hit, but we have better recall in four more classes. 
print(accuracy)
report_data = report.split('\n')
table = PrettyTable(["Category"] + report_data[0].split())
for line in report_data[2:-5]:
    table.add_row(line.split())
print(table)

In [ ]:
# multi-output regression
y_train_moutput = y_train[['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK',
       'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN']]
y_test_moutput = y_test[['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK',
       'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN']]

from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report


logistic_reg = LogisticRegression(penalty = "l1", solver='liblinear')
multioutput_classifier = MultiOutputClassifier(logistic_reg)
multioutput_classifier.fit(X_train_imputed, y_train_moutput)


y_pred = multioutput_classifier.predict(X_test_imputed)
print("Accuracy:", accuracy_score(y_test_moutput, y_pred))
print("Classification Report:")
print(classification_report(y_test_moutput, y_pred))

